In [1]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime
city_popu=pd.read_hdf('./Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

In [78]:
import os
dfo=pd.DataFrame()

for f in os.listdir('./DataSource/HeyTea/'):
    if any(i in f for i in ['2021-12']):
        print(f)
        date=f.split('_')[-1].split('.')[0]
        d=pd.read_csv('./DataSource/HeyTea\%s' % f)
        dfo=pd.concat([dfo,d],axis=0)

heytea.wxapp_2021-12-01.csv
heytea.wxapp_2021-12-02.csv
heytea.wxapp_2021-12-03.csv
heytea.wxapp_2021-12-04.csv
heytea.wxapp_2021-12-05.csv
heytea.wxapp_2021-12-06.csv
heytea.wxapp_2021-12-07.csv
heytea.wxapp_2021-12-08.csv
heytea.wxapp_2021-12-09.csv
heytea.wxapp_2021-12-10.csv
heytea.wxapp_2021-12-11.csv
heytea.wxapp_2021-12-12.csv
heytea.wxapp_2021-12-13.csv
heytea.wxapp_2021-12-14.csv
heytea.wxapp_2021-12-15.csv
heytea.wxapp_2021-12-16.csv
heytea.wxapp_2021-12-17.csv
heytea.wxapp_2021-12-18.csv
heytea.wxapp_2021-12-19.csv
heytea.wxapp_2021-12-20.csv
heytea.wxapp_2021-12-21.csv
heytea.wxapp_2021-12-22.csv
heytea.wxapp_2021-12-23.csv
heytea.wxapp_2021-12-24.csv
heytea.wxapp_2021-12-25.csv
heytea.wxapp_2021-12-26.csv
heytea.wxapp_2021-12-27.csv
heytea.wxapp_2021-12-28.csv
heytea.wxapp_2021-12-29.csv
heytea.wxapp_2021-12-30.csv
heytea.wxapp_2021-12-31.csv


In [79]:
old=pd.read_hdf(r'Processed_Data/heytea.h5',key='1227')
new=pd.concat([old,dfo])
new=new.drop_duplicates()
new.to_hdf(r'Processed_Data/heytea.h5',key='1227')

In [136]:
df=new.copy()
c1=df['抓取时间'].isin(['抓取时间'])
df=df[~c1]
df['datetime']=pd.to_datetime(df['抓取时间'])
df['hour']=df['datetime'].dt.hour
df['dt']=df['datetime'].dt.date
df['城市']=df['城市'].str.replace('市',"")

In [137]:
c2=df['dt']>=datetime.date(2022,1,1)
df=df[c2]

In [162]:
storelist=df.groupby(by=['hour','地址'])['ID'].count().reset_index().groupby(by='地址')['ID'].mean().reset_index()
storelist=storelist[storelist['ID']>350]['地址']
df=df[df['地址'].isin(storelist)]
df['等待杯量']=pd.to_numeric(df['等待杯量'])
# storelist

In [163]:
ht=df.groupby(by=['dt','城市'])['等待杯量'].sum().reset_index()
ht['杯量_last7d_mean']=ht.groupby('城市')['等待杯量'].transform(lambda x: x.rolling(7, 6).mean())

In [164]:
c1=ht['dt']< datetime.date(2022,3,1)
c2=ht['dt']> datetime.date(2022,2,6)
Mar_mean=ht[c1&c2].groupby('城市').mean().reset_index().sort_values(by='等待杯量')[['城市','等待杯量']].reset_index(drop=True)
Mar_mean.columns=['城市', '等待杯量_mar']
Mar_mean

ht=ht.merge(Mar_mean,on='城市')
ht['changes_since_mar']=ht['杯量_last7d_mean']/ht['等待杯量_mar']

In [169]:
pd.pivot_table(df,index=['城市'],columns='dt',values='等待杯量',aggfunc='sum').to_clipboard()

In [165]:
citylist=['北京','石家庄','上海','武汉']
c3=ht['城市'].isin(citylist)
c4=ht['dt']>datetime.date(2022,1,1)
show=ht[c3&c4]


show=pd.pivot_table(show,index='dt',columns='城市',values='changes_since_mar',aggfunc='mean').reset_index() #
show['sum']=np.nan

show=show[['dt','sum']+citylist]
show.to_clipboard(index=False)

In [166]:
df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
df_covid.tail(2)

,city,progress,peak,end,城市,常住人口,progress_2
295,聊城,回落至34%,2022/12/20,2023/03/07,聊城,201,34
296,宿迁,回落至41%,2022/12/21,2023/03/08,宿迁,310,41


In [167]:
# covid rank vs. HEYTEA WoW

%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
riqi='2022-12-26'

ht1=ht.merge(df_covid,left_on=['城市'],right_on=['城市'],how='left')
ht1['covid_rank']=ht1.groupby(by=['dt'])['progress_2'].rank(method="dense", ascending=False)
ht1['杯量_change_WoW']=(ht1['杯量_last7d_mean']/ht1.groupby('城市')['杯量_last7d_mean'].transform(lambda x: x.shift(periods=7)))-1


ht1['dt']=ht1['dt'].astype('string') 

ht2=ht1[ht1['dt']==riqi].sort_values(by='杯量_last7d_mean',ascending=False)
ht2=ht2[ht1['杯量_change_WoW']<10].head(100).reset_index()

o=ht2[['城市',  'covid_rank','杯量_change_WoW']]


o['size']=np.nan
o['group']='C'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='A'

c2=o['城市'].isin(jidui_city)
o.loc[c2,'group']='B'

o=o.sort_values(by='group')

o.to_clipboard(index=False,header=None)
print(riqi)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022-12-26


In [168]:
# covid rank vs. heytea vs 202202

ht1=ht.merge(df_covid,left_on=['城市'],right_on=['城市'],how='left')
ht1['covid_rank']=ht1.groupby(by=['dt'])['progress_2'].rank(method="dense", ascending=False)

ht1['dt']=ht1['dt'].astype('string') 

ht2=ht1[ht1['dt']==riqi].sort_values(by='杯量_last7d_mean',ascending=False)
ht2=ht2[ht1['changes_since_mar']<10].head(100).reset_index()
# hote1
o=ht2[['城市',  'covid_rank','changes_since_mar']]


o['size']=np.nan
o['group']='C'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='A'

c2=o['城市'].isin(jidui_city)
o.loc[c2,'group']='B'

o=o.sort_values(by='group')

o.to_clipboard(index=False,header=None)
print(riqi)

2022-12-26
